In [1]:
import sys
sys.path.append('/Users/bengikoseoglu/anaconda2/pkgs')
sys.path

['',
 '/Users/bengikoseoglu/anaconda2/lib/python27.zip',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/plat-darwin',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/plat-mac',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/plat-mac/lib-scriptpackages',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/lib-tk',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/lib-old',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/lib-dynload',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/aeosa',
 '/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/IPython/extensions',
 '/Users/bengikoseoglu/.ipython',
 '/Users/bengikoseoglu/anaconda2/pkgs']

In [2]:
import pandas as pd
import numpy as np
train=pd.read_csv('train.csv',delimiter='|')
prices=pd.read_csv('prices.csv',delimiter='|')
item=pd.read_csv('items.csv',delimiter='|') #need to replace with item

In [3]:
print (item.shape)
print (prices.shape)
print (train.shape)

(12824, 10)
(12824, 153)
(135117, 4)


## Step 1. Creating a primary key

In [4]:
train['key']=train['pid'].astype(str) +" "+train['size'].astype(str)
prices['key']=prices['pid'].astype(str) +" "+prices['size'].astype(str) 
item['key']=item['pid'].astype(str) +" "+item['size'].astype(str)

## Step 2. Fixing the variables

### 2.1. Size in items

#### 2.1.1: fill missing value: size, subCategory

In [5]:
null_columns=item.columns[item.isnull().any()]
item[null_columns].isnull().sum()

size              9
subCategory    1134
dtype: int64

In [6]:
size=item.drop(['subCategory'], axis=1)
null_size = size[size.isnull().any(axis=1)]
null_size

,pid,size,color,brand,rrp,mainCategory,category,stock,releaseDate,key
343,10400,NaN,schwarz,adidas,12.63,1,2,1,2017-10-01,10400 nan
540,10609,NaN,schwarz,Uhlsport,5.01,1,2,8,2017-10-01,10609 nan
3537,13515,NaN,schwarz,Uhlsport,2.47,1,2,1,2017-10-01,13515 nan
6050,15996,NaN,schwarz,Uhlsport,6.28,1,2,3,2017-10-01,15996 nan
8795,18754,NaN,schwarz,Uhlsport,2.47,1,2,2,2017-10-01,18754 nan
9495,19517,NaN,silber,adidas,15.17,1,2,1,2017-10-01,19517 nan
9894,19891,NaN,schwarz,Uhlsport,2.47,1,2,3,2017-10-01,19891 nan
10259,20248,NaN,weiss,Uhlsport,5.01,1,2,8,2017-10-01,20248 nan
10992,21001,NaN,silber,Uhlsport,8.82,1,2,1,2017-10-01,21001 nan


In [7]:
size_adidas= item[['size']].loc[item['brand'] == 'adidas']
most_frequent_size_adidas=size_adidas['size'].value_counts().idxmax()
most_frequent_size_adidas

'M'

In [8]:
size_Uhlsport= item[['size']].loc[item['brand'] == 'Uhlsport']
most_frequent_size_Uhlsport=size_Uhlsport['size'].value_counts().idxmax()
most_frequent_size_Uhlsport

'L'

In [9]:
item.loc[item['brand'] == 'adidas', 'size']=item['size'].fillna(most_frequent_size_adidas) # M
item.loc[item['brand'] == 'Uhlsport', 'size']=item['size'].fillna(most_frequent_size_Uhlsport) # L

item['subCategory']=item['subCategory'].fillna(0)

#### 2.1.2: unified size formate

Size map excel is created manually that matches sizes with the brand

In [13]:
item.head()

,pid,size,color,brand,rrp,mainCategory,category,subCategory,stock,releaseDate,key,brand_size
0,10000,XL ( 158-170 ),gruen,Nike,25.33,1,7,25.0,1,2017-10-01,10000 XL ( 158-170 ),Nike XL ( 158-170 )
1,10001,L,schwarz,Jako,38.03,1,7,16.0,1,2017-10-01,10001 L,Jako L
2,10003,3 (35-38 ),weiss,Jako,12.63,1,7,13.0,1,2017-10-01,10003 3 (35-38 ),Jako 3 (35-38 )
3,10003,4 ( 39-42 ),weiss,Jako,12.63,1,7,13.0,1,2017-10-01,10003 4 ( 39-42 ),Jako 4 ( 39-42 )
4,10003,5 ( 43-46 ),weiss,Jako,12.63,1,7,13.0,1,2017-10-01,10003 5 ( 43-46 ),Jako 5 ( 43-46 )


In [14]:
size_map.head()

,brand_size,brand,size,new size,pro_type
0,adidas 0 ( 31-33 ),adidas,0 ( 31-33 ),31-33,socks
1,adidas 00 ( 27-30 ),adidas,00 ( 27-30 ),27-30,socks
2,adidas 1 ( 34-36 ),adidas,1 ( 34-36 ),34-36,socks
3,adidas 116,adidas,116,116,kinder
4,adidas 128,adidas,128,128,kinder


In [11]:
size_map=pd.read_excel('size_map.xlsx')

In [12]:
item['brand_size']=item['brand'].astype(str)+" "+item['size'].astype(str)

In [15]:
size_map[['brand_size','new size','pro_type']].head()

,brand_size,new size,pro_type
0,adidas 0 ( 31-33 ),31-33,socks
1,adidas 00 ( 27-30 ),27-30,socks
2,adidas 1 ( 34-36 ),34-36,socks
3,adidas 116,116,kinder
4,adidas 128,128,kinder


In [17]:
item=pd.merge(size_map[['brand_size','new size','pro_type']],item, on='brand_size').drop(['brand_size'], axis=1)

In [19]:
print('brand unique values=%s'%len(item['brand'].unique()))
print('new size unique values=%s'%len(item['new size'].unique()))
print('mainCategory unique values=%s'%len(item['mainCategory'].unique()))
print('category unique values=%s'%len(item['category'].unique()))
print('subCategory unique values=%s'%len(item['subCategory'].unique()))
print('color unique values=%s'%len(item['color'].unique()))

brand unique values=25
new size unique values=105
mainCategory unique values=3
category unique values=10
subCategory unique values=33
color unique values=17


#### 2.1.3: handle categorical attributes

In [20]:
item['mainCategory'] = item['mainCategory'].astype(str)
item['category'] = item['category'].astype(str)
item['subCategory'] = item['subCategory'].astype(str)

item_dummy=item[['new size','color','brand','mainCategory','category','subCategory']]
item_normal=item[['key','pid','rrp','stock','releaseDate']]
item_dummy=pd.get_dummies(item_dummy)
item_pre = pd.concat([item_normal,item_dummy], axis=1)

## Step 3. Changing Format to Long

### 3.1. Prepare Prices Data

In [21]:
prices = pd.DataFrame(prices)
prices_=prices.drop(['pid', 'size'], axis=1)
prices_new = pd.melt(prices_, id_vars=["key"], 
                  var_name="date", value_name="price")
#prices_new= prices_.melt('key', var_name='date', value_name='price')  
a=pd.merge(prices, prices_new, on=['key'])
prices_change = a[['pid','size','price','date','key']]

prices_change.shape#(1936424, 5)

(1936424, 5)

In [22]:
prices_change['date']=pd.to_datetime(prices_change['date'])
n=prices_change.shape[0]

weekdays=[]
day=[]
month=[]
i=0

while(i<n):
    weekday_n=prices_change['date'][i].weekday()
    month_n=prices_change['date'][i].month
    day_n=prices_change['date'][i].day
    
    weekdays.append(weekday_n)
    day.append(day_n)
    month.append(month_n)
    i=i+1
    
prices_change['weekday']=weekdays
prices_change['day']=day
prices_change['month']=month

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [23]:
price_changes=prices_change[prices_change.month !=2 ] #Remove February

In [24]:
price_changes.head()

,pid,size,price,date,key,weekday,day,month
0,19671,39 1/3,133.31,2017-10-01,19671 39 1/3,6,1,10
1,19671,39 1/3,133.31,2017-10-02,19671 39 1/3,0,2,10
2,19671,39 1/3,133.31,2017-10-03,19671 39 1/3,1,3,10
3,19671,39 1/3,133.31,2017-10-04,19671 39 1/3,2,4,10
4,19671,39 1/3,133.31,2017-10-05,19671 39 1/3,3,5,10


In [25]:
item_selected=item_pre.copy()
data_regression10=pd.merge(price_changes[['key','weekday','day','month','date']],item_selected, on='key')
data_regression10['date']=pd.to_datetime(data_regression10['date'])
train['date']=pd.to_datetime(train['date'])
data_regression11=pd.merge(data_regression10, train[['key','date','units']], on=['key','date'],how='left')
data_regression11['units']=data_regression11['units'].fillna(0) # fill 0 to no sales

In [26]:
print(price_changes.shape)
print(data_regression11.shape)

(1577352, 8)
(1577352, 203)


In [27]:
price_changes.head()

,pid,size,price,date,key,weekday,day,month
0,19671,39 1/3,133.31,2017-10-01,19671 39 1/3,6,1,10
1,19671,39 1/3,133.31,2017-10-02,19671 39 1/3,0,2,10
2,19671,39 1/3,133.31,2017-10-03,19671 39 1/3,1,3,10
3,19671,39 1/3,133.31,2017-10-04,19671 39 1/3,2,4,10
4,19671,39 1/3,133.31,2017-10-05,19671 39 1/3,3,5,10


## Step 4. Adding Variables

### Data 1: new features: Holiday & Weather

- We find holidays and put into an excel and then combine it
- We also took weather data from https://www.accuweather.com/en/de/germany-weather

In [28]:
df_hw=pd.read_excel('holiday_weather.xlsx')

In [29]:
data_regression12=pd.merge(data_regression11, df_hw, on=['date'],how='left')

In [30]:
data_regression12.shape

(1577352, 207)

### Data 2: new features: Sum Units

This variable added, in case we would like to predict sum unit. 

In [31]:
data_regression12.head()

,key,weekday,day,month,date,pid,rrp,stock,releaseDate,new size_18,...,subCategory_43.0,subCategory_44.0,subCategory_5.0,subCategory_6.0,subCategory_8.0,units,avg_temp,median_temp,company_offer,holiday
0,19671 39 1/3,6,1,10,2017-10-01,19671,190.43,1,2017-10-01,0,...,0,0,0,0,0,0.0,12.5625,12.50,0,0
1,19671 39 1/3,0,2,10,2017-10-02,19671,190.43,1,2017-10-01,0,...,0,0,0,0,0,0.0,13.3125,13.75,0,0
2,19671 39 1/3,1,3,10,2017-10-03,19671,190.43,1,2017-10-01,0,...,0,0,0,0,0,0.0,12.1875,12.50,0,1
3,19671 39 1/3,2,4,10,2017-10-04,19671,190.43,1,2017-10-01,0,...,0,0,0,0,0,0.0,10.7500,10.75,0,0
4,19671 39 1/3,3,5,10,2017-10-05,19671,190.43,1,2017-10-01,0,...,0,0,0,0,0,1.0,11.7500,11.50,0,0


In [49]:
train['date']=pd.to_datetime(train['date'])
prices_change2=pd.merge(prices_change, train[['key','date','units']], on=['key','date'],how='left')#,'date'])
prices_change2['units']=prices_change2['units'].fillna(0)

/Users/bengikoseoglu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [51]:
count = len(prices_change2)
u=0

prices_change2['sum_unit']=0                     #iloc: 9


while u < count:
    
    if prices_change2.iloc[u-1]['key']!=prices_change2.iloc[u]['key']:  # first day of the new product  
        prices_change2.iloc[u,9]=prices_change2.iloc[u,8]  # iloc 15: units
            
    
    if prices_change2.iloc[u-1]['key']==prices_change2.iloc[u]['key']:
            
        if prices_change2.iloc[u-1]['month']!=prices_change2.iloc[u]['month']:  # first day of each month
            prices_change2.iloc[u,9]=prices_change2.iloc[u,8]
            
        if prices_change2.iloc[u-1]['month']==prices_change2.iloc[u]['month']:  # middle days in each month
            prices_change2.iloc[u,9]=prices_change2.iloc[u,8]+prices_change2.iloc[u-1,9]
    
    u+=1


### Data 3: new features: Price Daily Trend

Percentage price change

In [52]:
prices_change2['price_daily_change']='0.00'
i=1
while i<len(prices_change2):
    prices_change2.iloc[i,10]=(prices_change2.iloc[i,2]-prices_change2.iloc[i-1,2])/prices_change2.iloc[i-1,2]
    i=i+1

In [53]:
prices_change2.to_csv('prices_daily_trend.csv')

In [56]:
df_pridaily=pd.read_csv('prices_daily_trend.csv')

In [57]:
df_pridaily['price_daily_change']=df_pridaily['price_daily_change'].fillna(0)

In [58]:
df_pridaily['date']=pd.to_datetime(df_pridaily['date'])
data_regression14=pd.merge(data_regression12, df_pridaily[['key','date','sum_unit','price_daily_change']], on=['key','date'],how='left')

### Data 4: new features: New Product

Shows the day whether a product is new or not

In [60]:
from datetime import date, timedelta
d1 = date(2017, 10, 1)  # start date
d2 = date(2018, 1, 31)  # end date
d3=date(2018, 1, 31)
delta = d3 - d1         # timedelta
pre_day=[]
for i in range(delta.days + 1):
    add=str(d1 + timedelta(days=i))
    pre_day.append(add)
df_days_pre = pd.DataFrame({'date':pre_day})

In [62]:
all_1=item.key.unique()
n=len(item.key.unique())
i=0
k=0
n1=len(pre_day)
days=[]
ids=[]
rel_date=[]
while(i<n):
    b=all_1[i]
    r=item[item['key']==all_1[i]]['releaseDate'].values[0]
    k=0
    while(k<n1):
        a=pre_day[k]
        ids.append(b)
        days.append(a)
        rel_date.append(r)
        k=k+1
    i=i+1

In [63]:
train_days = pd.DataFrame({'date':days,'key':ids,'release_date':rel_date})

In [64]:
train=train[['units','key','date']]

In [65]:
train_comb=train_days.merge(train,on=['date','key'],how='left')
train_comb=train_comb.fillna(0)

In [66]:
train_comb['release_date'] = pd.to_datetime(train_comb['release_date'])

In [67]:
train_comb['date'] = pd.to_datetime(train_comb['date'])

In [68]:
all_=train_comb.key.unique()
i=0
n=len(all_)
price_change_date=[]
while(i<n):
    print(len(price_change_date))
    #data=data_s.copy()
    data=train_comb[train_comb['key']==all_[i]]
    data=data.reset_index()
    a=data.shape[0]
    k=0
    if(data['release_date'][0]==data['date'][0]):
        p=0
        while(p<a):
            price_change_date.append('0')
            p=p+1
    else:
        while(k<a):
            if(data['release_date'][k]>data['date'][k]):
                price_change_date.append('0')
            else:
                p=0
                a=a-k
                while(p<a):
                    price_change_date.append('1')
                    p=p+1
                break
            k=k+1
    i=i+1

0
123
246
369
492
615
738
861
984
1107
1230
1353
1476
1599
1722
1845
1968
2091
2214
2337
2460
2583
2706
2829
2952
3075
3198
3321
3444
3567
3690
3813
3936
4059
4182
4305
4428
4551
4674
4797
4920
5043
5166
5289
5412
5535
5658
5781
5904
6027
6150
6273
6396
6519
6642
6765
6888
7011
7134
7257
7380
7503
7626
7749
7872
7995
8118
8241
8364
8487
8610
8733
8856
8979
9102
9225
9348
9471
9594
9717
9840
9963
10086
10209
10332
10455
10578
10701
10824
10947
11070
11193
11316
11439
11562
11685
11808
11931
12054
12177
12300
12423
12546
12669
12792
12915
13038
13161
13284
13407
13530
13653
13776
13899
14022
14145
14268
14391
14514
14637
14760
14883
15006
15129
15252
15375
15498
15621
15744
15867
15990
16113
16236
16359
16482
16605
16728
16851
16974
17097
17220
17343
17466
17589
17712
17835
17958
18081
18204
18327
18450
18573
18696
18819
18942
19065
19188
19311
19434
19557
19680
19803
19926
20049
20172
20295
20418
20541
20664
20787
20910
21033
21156
21279
21402
21525
21648
21771
21894
22017
22140
22263
2

160146
160269
160392
160515
160638
160761
160884
161007
161130
161253
161376
161499
161622
161745
161868
161991
162114
162237
162360
162483
162606
162729
162852
162975
163098
163221
163344
163467
163590
163713
163836
163959
164082
164205
164328
164451
164574
164697
164820
164943
165066
165189
165312
165435
165558
165681
165804
165927
166050
166173
166296
166419
166542
166665
166788
166911
167034
167157
167280
167403
167526
167649
167772
167895
168018
168141
168264
168387
168510
168633
168756
168879
169002
169125
169248
169371
169494
169617
169740
169863
169986
170109
170232
170355
170478
170601
170724
170847
170970
171093
171216
171339
171462
171585
171708
171831
171954
172077
172200
172323
172446
172569
172692
172815
172938
173061
173184
173307
173430
173553
173676
173799
173922
174045
174168
174291
174414
174537
174660
174783
174906
175029
175152
175275
175398
175521
175644
175767
175890
176013
176136
176259
176382
176505
176628
176751
176874
176997
177120
177243
177366
177489
177612

304425
304548
304671
304794
304917
305040
305163
305286
305409
305532
305655
305778
305901
306024
306147
306270
306393
306516
306639
306762
306885
307008
307131
307254
307377
307500
307623
307746
307869
307992
308115
308238
308361
308484
308607
308730
308853
308976
309099
309222
309345
309468
309591
309714
309837
309960
310083
310206
310329
310452
310575
310698
310821
310944
311067
311190
311313
311436
311559
311682
311805
311928
312051
312174
312297
312420
312543
312666
312789
312912
313035
313158
313281
313404
313527
313650
313773
313896
314019
314142
314265
314388
314511
314634
314757
314880
315003
315126
315249
315372
315495
315618
315741
315864
315987
316110
316233
316356
316479
316602
316725
316848
316971
317094
317217
317340
317463
317586
317709
317832
317955
318078
318201
318324
318447
318570
318693
318816
318939
319062
319185
319308
319431
319554
319677
319800
319923
320046
320169
320292
320415
320538
320661
320784
320907
321030
321153
321276
321399
321522
321645
321768
321891

448581
448704
448827
448950
449073
449196
449319
449442
449565
449688
449811
449934
450057
450180
450303
450426
450549
450672
450795
450918
451041
451164
451287
451410
451533
451656
451779
451902
452025
452148
452271
452394
452517
452640
452763
452886
453009
453132
453255
453378
453501
453624
453747
453870
453993
454116
454239
454362
454485
454608
454731
454854
454977
455100
455223
455346
455469
455592
455715
455838
455961
456084
456207
456330
456453
456576
456699
456822
456945
457068
457191
457314
457437
457560
457683
457806
457929
458052
458175
458298
458421
458544
458667
458790
458913
459036
459159
459282
459405
459528
459651
459774
459897
460020
460143
460266
460389
460512
460635
460758
460881
461004
461127
461250
461373
461496
461619
461742
461865
461988
462111
462234
462357
462480
462603
462726
462849
462972
463095
463218
463341
463464
463587
463710
463833
463956
464079
464202
464325
464448
464571
464694
464817
464940
465063
465186
465309
465432
465555
465678
465801
465924
466047

736770
736893
737016
737139
737262
737385
737508
737631
737754
737877
738000
738123
738246
738369
738492
738615
738738
738861
738984
739107
739230
739353
739476
739599
739722
739845
739968
740091
740214
740337
740460
740583
740706
740829
740952
741075
741198
741321
741444
741567
741690
741813
741936
742059
742182
742305
742428
742551
742674
742797
742920
743043
743166
743289
743412
743535
743658
743781
743904
744027
744150
744273
744396
744519
744642
744765
744888
745011
745134
745257
745380
745503
745626
745749
745872
745995
746118
746241
746364
746487
746610
746733
746856
746979
747102
747225
747348
747471
747594
747717
747840
747963
748086
748209
748332
748455
748578
748701
748824
748947
749070
749193
749316
749439
749562
749685
749808
749931
750054
750177
750300
750423
750546
750669
750792
750915
751038
751161
751284
751407
751530
751653
751776
751899
752022
752145
752268
752391
752514
752637
752760
752883
753006
753129
753252
753375
753498
753621
753744
753867
753990
754113
754236

880926
881049
881172
881295
881418
881541
881664
881787
881910
882033
882156
882279
882402
882525
882648
882771
882894
883017
883140
883263
883386
883509
883632
883755
883878
884001
884124
884247
884370
884493
884616
884739
884862
884985
885108
885231
885354
885477
885600
885723
885846
885969
886092
886215
886338
886461
886584
886707
886830
886953
887076
887199
887322
887445
887568
887691
887814
887937
888060
888183
888306
888429
888552
888675
888798
888921
889044
889167
889290
889413
889536
889659
889782
889905
890028
890151
890274
890397
890520
890643
890766
890889
891012
891135
891258
891381
891504
891627
891750
891873
891996
892119
892242
892365
892488
892611
892734
892857
892980
893103
893226
893349
893472
893595
893718
893841
893964
894087
894210
894333
894456
894579
894702
894825
894948
895071
895194
895317
895440
895563
895686
895809
895932
896055
896178
896301
896424
896547
896670
896793
896916
897039
897162
897285
897408
897531
897654
897777
897900
898023
898146
898269
898392

1021884
1022007
1022130
1022253
1022376
1022499
1022622
1022745
1022868
1022991
1023114
1023237
1023360
1023483
1023606
1023729
1023852
1023975
1024098
1024221
1024344
1024467
1024590
1024713
1024836
1024959
1025082
1025205
1025328
1025451
1025574
1025697
1025820
1025943
1026066
1026189
1026312
1026435
1026558
1026681
1026804
1026927
1027050
1027173
1027296
1027419
1027542
1027665
1027788
1027911
1028034
1028157
1028280
1028403
1028526
1028649
1028772
1028895
1029018
1029141
1029264
1029387
1029510
1029633
1029756
1029879
1030002
1030125
1030248
1030371
1030494
1030617
1030740
1030863
1030986
1031109
1031232
1031355
1031478
1031601
1031724
1031847
1031970
1032093
1032216
1032339
1032462
1032585
1032708
1032831
1032954
1033077
1033200
1033323
1033446
1033569
1033692
1033815
1033938
1034061
1034184
1034307
1034430
1034553
1034676
1034799
1034922
1035045
1035168
1035291
1035414
1035537
1035660
1035783
1035906
1036029
1036152
1036275
1036398
1036521
1036644
1036767
1036890
1037013
1037136


1148082
1148205
1148328
1148451
1148574
1148697
1148820
1148943
1149066
1149189
1149312
1149435
1149558
1149681
1149804
1149927
1150050
1150173
1150296
1150419
1150542
1150665
1150788
1150911
1151034
1151157
1151280
1151403
1151526
1151649
1151772
1151895
1152018
1152141
1152264
1152387
1152510
1152633
1152756
1152879
1153002
1153125
1153248
1153371
1153494
1153617
1153740
1153863
1153986
1154109
1154232
1154355
1154478
1154601
1154724
1154847
1154970
1155093
1155216
1155339
1155462
1155585
1155708
1155831
1155954
1156077
1156200
1156323
1156446
1156569
1156692
1156815
1156938
1157061
1157184
1157307
1157430
1157553
1157676
1157799
1157922
1158045
1158168
1158291
1158414
1158537
1158660
1158783
1158906
1159029
1159152
1159275
1159398
1159521
1159644
1159767
1159890
1160013
1160136
1160259
1160382
1160505
1160628
1160751
1160874
1160997
1161120
1161243
1161366
1161489
1161612
1161735
1161858
1161981
1162104
1162227
1162350
1162473
1162596
1162719
1162842
1162965
1163088
1163211
1163334


1274280
1274403
1274526
1274649
1274772
1274895
1275018
1275141
1275264
1275387
1275510
1275633
1275756
1275879
1276002
1276125
1276248
1276371
1276494
1276617
1276740
1276863
1276986
1277109
1277232
1277355
1277478
1277601
1277724
1277847
1277970
1278093
1278216
1278339
1278462
1278585
1278708
1278831
1278954
1279077
1279200
1279323
1279446
1279569
1279692
1279815
1279938
1280061
1280184
1280307
1280430
1280553
1280676
1280799
1280922
1281045
1281168
1281291
1281414
1281537
1281660
1281783
1281906
1282029
1282152
1282275
1282398
1282521
1282644
1282767
1282890
1283013
1283136
1283259
1283382
1283505
1283628
1283751
1283874
1283997
1284120
1284243
1284366
1284489
1284612
1284735
1284858
1284981
1285104
1285227
1285350
1285473
1285596
1285719
1285842
1285965
1286088
1286211
1286334
1286457
1286580
1286703
1286826
1286949
1287072
1287195
1287318
1287441
1287564
1287687
1287810
1287933
1288056
1288179
1288302
1288425
1288548
1288671
1288794
1288917
1289040
1289163
1289286
1289409
1289532


1526553
1526676
1526799
1526922
1527045
1527168
1527291
1527414
1527537
1527660
1527783
1527906
1528029
1528152
1528275
1528398
1528521
1528644
1528767
1528890
1529013
1529136
1529259
1529382
1529505
1529628
1529751
1529874
1529997
1530120
1530243
1530366
1530489
1530612
1530735
1530858
1530981
1531104
1531227
1531350
1531473
1531596
1531719
1531842
1531965
1532088
1532211
1532334
1532457
1532580
1532703
1532826
1532949
1533072
1533195
1533318
1533441
1533564
1533687
1533810
1533933
1534056
1534179
1534302
1534425
1534548
1534671
1534794
1534917
1535040
1535163
1535286
1535409
1535532
1535655
1535778
1535901
1536024
1536147
1536270
1536393
1536516
1536639
1536762
1536885
1537008
1537131
1537254
1537377
1537500
1537623
1537746
1537869
1537992
1538115
1538238
1538361
1538484
1538607
1538730
1538853
1538976
1539099
1539222
1539345
1539468
1539591
1539714
1539837
1539960
1540083
1540206
1540329
1540452
1540575
1540698
1540821
1540944
1541067
1541190
1541313
1541436
1541559
1541682
1541805


In [69]:
train_comb['new_product']=price_change_date

In [70]:
train_comb.to_csv('new_product.csv')

In [71]:
df_nproduct=pd.read_csv('new_product.csv')

In [72]:
df_nproduct['date']=pd.to_datetime(df_nproduct['date'])
data_regression14=pd.merge(data_regression14, df_nproduct[['key','date','new_product']], on=['key','date'],how='left')

In [73]:
data_regression14.shape

(1577352, 210)

### Data 5: new features: Remove Black Friday

In [75]:
data = data_regression14.drop(data_regression14[(data_regression14.month==11) & (data_regression14.day == 24)].index)

### Export

In [ ]:
data.to_csv('integrated_data.csv')